# b) Ngonye Falls Synthetic Historic Flow Series

To produce a long time series (1924-2017) of daily flow for the Ngonye Falls site, a correlation is carried out between the overlapping portions of the *Zambezi River Authority* (ZRA) gauge at Ngonye Falls and portion of the record of the Victoria Falls gauge (2005-2016).

Once correlated, the full Victoria Falls gauge record is extrapolated to produce a synthetic record for Ngonye Falls.

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

1. Calculate Flow Duration Curves (FDCs) from daily data for the following:
    1. The target site (Ngonye) for 2005/06 – 2016/17; 
    2. The analogue site (Victoria Falls) for the exact period of data at the target site (2005/06 – 2016/17);
    3. Analogue site for long-term record (1924/25 -2016/17); 
    
    
2. Compare the FDCs for the analogue site for the two periods and derive factors between the shorter and longer periods; 

3. Apply these factors to the target site FDC to produce an estimated long-term FDC for the target site;

4. For each daily flow value at the analogue site, determine its position on the long-term FDC (percentile); and,

5. Look up the flow value for this percentile from the estimated long-term FDC for the target site taking into account of the 11-day time lag.

*Water Year* is defined as running Oct to Sept as is usual in the region.

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| daily_gauge_vicfalls.csv        | Zambezi River Authority | Daily Flow for Victoria Falls 1924/25 - 2016/17              |
| daily_gauge_ngonye.csv | Zambezi River Authority   | Daily Stage (Level) for Ngonye Falls 2005/06 – 2016/17 |
|  Stage - Discharge equation for Ngonye Falls |  Mott MacDonald |Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D |


## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_synthetic.csv  | Synthetic daily flow series for Ngonye  Falls 1924/25 - 2016/17  |
| ngonye_vicfalls_fdcs.csv           | Flow Duration Curves for Victoria Falls and Ngonye Falls |


## Parameters

In [295]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [296]:
import numpy as np
import pandas as pd
import datetime

## Load Data
Load the Vic Falls data and add some helper columns.

In [297]:
vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls.csv")

vicfalls['Date']=pd.to_datetime(vicfalls['Date'],format="%d/%m/%Y")
vicfalls=vicfalls.set_index(pd.DatetimeIndex(vicfalls['Date']))
vicfalls=vicfalls.drop(['Date'],axis=1)

vicfalls=vicfalls.astype({'Flow': 'float64'})
vicfalls['Flow']=np.round(vicfalls['Flow'],4)

vicfalls.head(5)

,Flow
Date,
1924-10-01,100.0
1924-10-02,100.0
1924-10-03,100.0
1924-10-04,100.0
1924-10-05,100.0


Load the Ngonye Falls level data and calculate flow based on the stage-discharge relationship:

\begin{equation*}
flow=1093.0355*(level-2.85)^{1.659}
\end{equation*}

Add some helper columns.

In [298]:
ngo=pd.read_csv(input_data+ "daily_gauge_ngonye.csv")
ngo['Date']=pd.to_datetime(ngo['Date'],format="%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo=ngo.drop(['Date'],axis=1)
ngo['Flow']=np.round(1093.0355*(ngo['Level']-2.85)**1.659,4)
ngo['VicFalls']=vicfalls['Flow']
ngo.head(5)

,Level,Flow,VicFalls
Date,,,
2005-10-01,3.218,208.1497,211.8138
2005-10-02,3.216,206.2763,210.5129
2005-10-03,3.215,205.3422,208.5712
2005-10-04,3.213,203.4789,204.0855
2005-10-05,3.212,202.5498,204.0855


## Flow Duration Curve

Build the Flow Duration Curve table in 0.1% exceedance increments.

Add FDC flows for the full Vic Falls timeseries.

In [299]:
fdc=pd.DataFrame({'Exceedance': np.arange(0,1.001,0.001)}).set_index('Exceedance')
fdc['VicFalls_full']=np.round(np.quantile(vicfalls.loc[vicfalls.index<=datetime.datetime(2017, 9,30)]['Flow'],((1-fdc.index))),4)
fdc

,VicFalls_full
Exceedance,
0.000,9435.5576
0.001,8466.2230
0.002,7545.5100
0.003,6745.6244
0.004,6292.9208
...,...
0.996,128.1598
0.997,115.2667
0.998,107.1446


Add flows for Ngonye to the FDC.

Add flows for the portion of the Vic Falls series that overlaps with the Ngonye Series.

In [300]:
fdc['Ngonye_gauged']=np.round(np.percentile(ngo.loc[ngo.index<=datetime.datetime(2017, 9,30)]['Flow'],((1-fdc.index)*100)),4)
fdc['VicFalls_overlap']=np.round(np.percentile(ngo.loc[ngo.index<=datetime.datetime(2017, 9,30)]['VicFalls'],((1-fdc.index)*100)),4)

## Vic Falls Ratio
Calculate the ratio of Vic Falls flows for the whole series and the overallping protion across the FDC.

Smooth that ratio (0.7% moving average) except at the tails of the FDC.

In [301]:
fdc['VicFalls_factor']=fdc['VicFalls_full']/fdc['VicFalls_overlap']
fdc['VicFalls_factor_smooth']=fdc['VicFalls_factor'].rolling(7,center=True).mean()
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name<0.01 else x['VicFalls_factor_smooth'])),axis=1)
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name>0.99 else x['VicFalls_factor_smooth'])),axis=1)

## Scaled FDC
Produce a scaled FDC for Ngonye by using the factors calculated for the Vic Falls data.

In [302]:
fdc['Ngonye_scaled']=fdc['Ngonye_gauged']*fdc['VicFalls_factor_smooth']

## Conversion
Calculate a conversion factor for each row of the FDC between the scaled Ngonye FDC flows and the full Vic Falls series FDC.

In [303]:
fdc['Conversion']=fdc['Ngonye_scaled']/fdc['VicFalls_full']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion
Exceedance,,,,,,,
0.000,9435.5576,5935.9208,5650.5399,1.669851,1.669851,9912.101075,1.050505
0.001,8466.2230,5830.0826,5590.0873,1.514506,1.514506,8829.697418,1.042932
0.002,7545.5100,5724.9991,5499.6002,1.372011,1.372011,7854.759689,1.040985
0.003,6745.6244,5575.5019,5329.7868,1.265646,1.265646,7056.612782,1.046102
0.004,6292.9208,5462.0358,5264.0282,1.195457,1.195457,6529.630426,1.037615
...,...,...,...,...,...,...,...
0.996,128.1598,167.2431,154.4870,0.829583,0.829583,138.742045,1.082571
0.997,115.2667,165.9305,151.8414,0.759126,0.759126,125.962097,1.092788
0.998,107.1446,164.2189,146.6380,0.730674,0.730674,119.990510,1.119893


Lookup the conversion factors from the FDC based on flow and give each record in the full Vic Falls series its corresponding conversion factor. 

In [304]:
if 'Conversion' in vicfalls.columns:
    display(vicfalls.columns)
    vicfalls=vicfalls.drop(['Conversion','Exceedance'],axis=1)

#vicfalls['tmp_Flow']=100000-vicfalls['Flow']
#fdc['tmp_Flow']=100000-fdc['VicFalls_full']
fdc['ExceedanceInv']=1-fdc.index

tmp=pd.merge_asof(vicfalls.reset_index().sort_values('Flow',ascending=True),fdc.reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=True),left_on='Flow',right_on='VicFalls_full').set_index('Date')
vicfalls['Conversion']=tmp['Conversion']
vicfalls['Exceedance']=tmp['Exceedance']



## Ngonye Sythetic
Prepare the full  synthetic series for Ngonye by applying the 11 day lag to the Vic Falls series and the conversion factors calculated previoulsy.

In [305]:

ngonye_synth=pd.DataFrame(index=vicfalls.index)

ngonye_synth['LaggedDate']=ngonye_synth.index+pd.DateOffset(days=11)
ngonye_synth['VicFalls']=ngonye_synth.join(vicfalls,on='LaggedDate')['Flow']
ngonye_synth['Conversion']=ngonye_synth.join(vicfalls,on='LaggedDate')['Conversion']
ngonye_synth['Flow']=ngonye_synth['Conversion'] * ngonye_synth['VicFalls']

vicfalls.loc[vicfalls.index==datetime.datetime(1924,12,16)]

,Flow,Conversion,Exceedance
Date,,,
1924-12-16,231.0479,0.9724,0.917


Remove extra columns and delete from the bottom to align to the water year.

In [306]:
ngonye_synth=ngonye_synth.dropna()
ngonye_synth=ngonye_synth.drop(ngonye_synth.loc['2017-10-01':].index)
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0000,1.114075,111.407480
1924-10-02,1924-10-13,100.0000,1.114075,111.407480
1924-10-03,1924-10-14,100.0000,1.114075,111.407480
1924-10-04,1924-10-15,100.0000,1.114075,111.407480
1924-10-05,1924-10-16,100.0000,1.114075,111.407480
...,...,...,...,...
2017-09-26,2017-10-07,204.0855,0.982689,200.552607
2017-09-27,2017-10-08,204.0855,0.982689,200.552607
2017-09-28,2017-10-09,204.0855,0.982689,200.552607


Add the new synthetic Ngonye flow series to the FDC.

In [307]:
fdc['Ngonye_synthetic']=np.percentile(ngonye_synth['Flow'],((1-fdc.index)*100))

tmp=pd.merge_asof(ngonye_synth.sort_values('Flow').reset_index(),fdc.sort_values('Ngonye_synthetic').reset_index(),left_on='Flow',right_on='Ngonye_synthetic').set_index('Date')
ngonye_synth['Exceedance']=tmp['Exceedance']

# Prepare Export
Prepare a subset of the FDC for export

In [308]:
fdc_out=fdc.loc[:,['VicFalls_full','Ngonye_gauged','Ngonye_synthetic','Conversion']]
fdc_out

,VicFalls_full,Ngonye_gauged,Ngonye_synthetic,Conversion
Exceedance,,,,
0.000,9435.5576,5935.9208,9912.101075,1.050505
0.001,8466.2230,5830.0826,8829.697418,1.042932
0.002,7545.5100,5724.9991,7890.806381,1.040985
0.003,6745.6244,5575.5019,7005.060692,1.046102
0.004,6292.9208,5462.0358,6479.994666,1.037615
...,...,...,...,...
0.996,128.1598,167.2431,142.406007,1.082571
0.997,115.2667,165.9305,130.554212,1.092788
0.998,107.1446,164.2189,124.486097,1.119893


## Export

In [309]:
ngonye_synth.to_csv(output_data + 'ngonye_synthetic.csv')
fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs.csv')

In [310]:
ngonye_synth.loc[ngonye_synth.index=='1925-02-05']

,LaggedDate,VicFalls,Conversion,Flow,Exceedance
Date,,,,,
1925-02-05,1925-02-16,1238.2939,0.914383,1132.274462,0.29


In [311]:
fdc.loc[(fdc.index<=0.291) & (fdc.index>=0.288)].reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=[True,True])

,Exceedance,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion,ExceedanceInv,Ngonye_synthetic
3,0.291,1234.8221,1553.0967,1694.0502,0.728917,0.728409,1131.288882,0.916155,0.709,1131.387162
2,0.290,1238.2939,1558.0660,1708.4979,0.724785,0.726718,1132.274462,0.914383,0.710,1132.274462
1,0.289,1244.9702,1561.7835,1708.4979,0.728693,0.726151,1134.090397,0.910938,0.711,1134.090397
0,0.288,1247.5890,1567.1999,1720.7979,0.725006,0.725370,1136.799050,0.911197,0.712,1141.726989
